# Part 2: Data Cleaning

In this part I will remove all unnecessary columns, and I will add necessary columns.
The columns that will be after this part are:
* Scale
* years
* Length
* Width
* Country
* Latitude
* Longtitude
* Deaths

Import necessary packages

In [1]:
import pandas as pd
import numpy as np

**This function removes all unnecessary columns**

In [2]:
def removeColumns(df, columnsToRemove):
    for column in columnsToRemove:
        df.drop(column, inplace=True, axis=1)
    return df

**These functions handling with missing data**

In [3]:
# This function removes all rows that contain an unkown scale (EFU) 
def removeRowsWithEFU(df):
    df = df[df["Scale"].str.contains("EFU") == False]
    return df

In [4]:
# This function removes all rows that contain NaN value
def removeRowsWithNaN(df):
    df = df.dropna(axis=0, how='any')
    return df

**This function handling with duplicates**

In [5]:
# This function removes the rows that appear more than once and leaves one copy
def removeDuplicates(df):
    df = df.drop_duplicates()
    return df

**Add NaN columns to create a new column**

In [6]:
def addNaNColumns(df, place_number, column_name):
    df.insert(loc=place_number, column=column_name, value=np.nan)
    return df

**Handling with columns that contain two different values in one column**

In [7]:
# This function takes the death direct value from Deaths Direct/Indirect column
def fixedDeathsColumn(df, numberOfRow):
    value = df.iloc[numberOfRow]['Deaths Direct/Indirect']
    deaths = value[0]
    return deaths

In [8]:
# This function fill the the death column with the values that we took from the Deaths Direct/Indirect column
def fillDeaths(df):
    for numberOfRow in range(len(df)):
        deaths = fixedDeathsColumn(df, numberOfRow)
        df.loc[df.index[numberOfRow], 'Deaths'] = deaths
    return df

In [9]:
# This function take care of the Begin Lat/Lon column and return the latitude and longtitude values separately
def fixedLatAndLon(df, numberOfRow):
    longtitude = list()
    latitude = list()
    flag = 0
    value = df.iloc[numberOfRow]['Begin Lat/Lon']
    for x in value:
        if x != '/' and flag == 0:
            latitude.extend(x)
        if x == '/':
            flag = 1
            continue
        if flag == 1:
            longtitude.extend(x)

    return [latitude, longtitude]

In [10]:
# This function fill the latitude and longtitude columns with the values that we took from the Begin Lat/Lon column
def fillLatAndLonColumns(df):
    for numberOfRow in range(len(df)):
        Latitude = ' '
        Longtitude = ' '
        listLatLon = fixedLatAndLon(df, numberOfRow)
        Type_new1 = listLatLon[0]
        Type_new2 = listLatLon[1]
        Latitude = ''.join(map(str, Type_new1))
        Longtitude = ''.join(map(str, Type_new2))
        df.loc[df.index[numberOfRow], 'Latitude'] = Latitude
        df.loc[df.index[numberOfRow], 'Longtitude'] = Longtitude
    return df

In [11]:
# This function take care of Begin Date column and return the year value
def fixedDate(df, number_of_row):
    date = df.iloc[number_of_row]['Begin Date']
    fixedDateList = list()
    for i in date:
        if i == '-':
            break
        else:
            fixedDateList.extend(i)

    fixedDate = ''.join(map(str, fixedDateList))
    return fixedDate

In [12]:
# This function fill the years column with the value that we took from the Begin Date column
def fillColumnYears(df):
    for numberOfRow in range(len(df)):
        year = fixedDate(df, numberOfRow)
        df.loc[df.index[numberOfRow], 'Years'] = year
    return df

**Handling with different units of measurement**

In [13]:
# This function convert yards to meters in width column
def convertYardsToMeters(df):
    value1 = list()
    for row_number in range(len(df)):
        value1.clear()
        value = df.iloc[row_number]['Width']
        for i in value:
            if i == ' ':
                break
            else:
                value1.extend(i)
        value = ''.join(map(str, value1))
        value = float(value) * 0.0009144
        limited_value = round(value, 3)
        df.loc[df.index[row_number], 'Width'] = limited_value
    return df

In [14]:
# This function convert miles to km in length column
def convertMilesToKm(df):
    value1 = list()
    for row_number in range(len(df)):
        value1.clear()
        value = df.iloc[row_number]['Length']
        for i in value:
            if i == ' ':
                break
            else:
                value1.extend(i)
        value = ''.join(map(str, value1))
        value = float(value) * 1.609344
        limited_value = round(value, 3)
        df.loc[df.index[row_number], 'Length'] = limited_value
    return df

In [15]:
# This function convert the scale value from EF to number 
def convertScaleToNumbers(df):
    for numberOfRow in range(len(df)):
        value = df.iloc[numberOfRow]['Scale']
        for i in value:
            if i != 'E' or i != 'F':
                value = i
        df.loc[df.index[numberOfRow], 'Scale'] = value
    return df

# main

**The df before data cleaning:**

In [16]:
df = pd.read_csv("TornadoTable.csv")
df

,Scale,Length,Width,Country,Begin Date,Begin Lat/Lon,Deaths Direct/Indirect
0,EF1,2.01 Miles,100 Yards,PENNSYLVANIA,2022-03-31 20:48 EST-5,40.379/-75.2266,"0/0 (fatality details below, when available...)"
1,EF0,0.13 Miles,50 Yards,VIRGINIA,2022-03-31 19:41 EST-5,38.9196/-77.2259,"0/0 (fatality details below, when available...)"
2,EF0,0.05 Miles,30 Yards,VIRGINIA,2022-03-31 19:22 EST-5,38.8653/-77.4579,"0/0 (fatality details below, when available...)"
3,EF1,9.14 Miles,500 Yards,PENNSYLVANIA,2022-03-31 15:38 EST-5,41.16/-76.73,"1/0 (fatality details below, when available...)"
4,EF1,0.22 Miles,100 Yards,NORTH CAROLINA,2022-03-31 14:49 EST-5,35.8672/-78.8372,"0/0 (fatality details below, when available...)"
...,...,...,...,...,...,...,...
15844,EF1,0.79 Miles,150 Yards,TEXAS,2010-01-20 16:26 CST-6,31.5095/-93.9232,"0/0 (fatality details below, when available...)"
15845,EF0,0.16 Miles,25 Yards,LOUISIANA,2010-01-20 15:27 CST-6,30.815/-92.0455,"0/0 (fatality details below, when available...)"
15846,EF1,4.62 Miles,25 Yards,CALIFORNIA,2010-01-19 12:55 PST-8,33.716/-118.125,"0/0 (fatality details below, when available...)"
15847,EF0,0.14 Miles,10 Yards,CALIFORNIA,2010-01-19 10:32 PST-8,34.4248/-119.877,"0/0 (fatality details below, when available...)"


**The df after removes rows with NaN values and duplicates:**

In [17]:
df = removeRowsWithEFU(df)
df = removeRowsWithNaN(df)
df = removeDuplicates(df)
df

,Scale,Length,Width,Country,Begin Date,Begin Lat/Lon,Deaths Direct/Indirect
0,EF1,2.01 Miles,100 Yards,PENNSYLVANIA,2022-03-31 20:48 EST-5,40.379/-75.2266,"0/0 (fatality details below, when available...)"
1,EF0,0.13 Miles,50 Yards,VIRGINIA,2022-03-31 19:41 EST-5,38.9196/-77.2259,"0/0 (fatality details below, when available...)"
2,EF0,0.05 Miles,30 Yards,VIRGINIA,2022-03-31 19:22 EST-5,38.8653/-77.4579,"0/0 (fatality details below, when available...)"
3,EF1,9.14 Miles,500 Yards,PENNSYLVANIA,2022-03-31 15:38 EST-5,41.16/-76.73,"1/0 (fatality details below, when available...)"
4,EF1,0.22 Miles,100 Yards,NORTH CAROLINA,2022-03-31 14:49 EST-5,35.8672/-78.8372,"0/0 (fatality details below, when available...)"
...,...,...,...,...,...,...,...
15844,EF1,0.79 Miles,150 Yards,TEXAS,2010-01-20 16:26 CST-6,31.5095/-93.9232,"0/0 (fatality details below, when available...)"
15845,EF0,0.16 Miles,25 Yards,LOUISIANA,2010-01-20 15:27 CST-6,30.815/-92.0455,"0/0 (fatality details below, when available...)"
15846,EF1,4.62 Miles,25 Yards,CALIFORNIA,2010-01-19 12:55 PST-8,33.716/-118.125,"0/0 (fatality details below, when available...)"
15847,EF0,0.14 Miles,10 Yards,CALIFORNIA,2010-01-19 10:32 PST-8,34.4248/-119.877,"0/0 (fatality details below, when available...)"


**The df after remove all unnecessary columns, and add all necessary columns:**

In [18]:
df = addNaNColumns(df, len(df.columns), 'Latitude')
df = addNaNColumns(df, len(df.columns), 'Longtitude')
df = addNaNColumns(df, len(df.columns), 'Deaths')
df = addNaNColumns(df, 1, 'Years')
df = fillDeaths(df)
df = fillLatAndLonColumns(df)
df = fillColumnYears(df)
columns_to_remove = ['Begin Lat/Lon', 'Deaths Direct/Indirect', 'Begin Date']
df = removeColumns(df, columns_to_remove)
df

,Scale,Years,Length,Width,Country,Latitude,Longtitude,Deaths
0,EF1,2022,2.01 Miles,100 Yards,PENNSYLVANIA,40.379,-75.2266,0
1,EF0,2022,0.13 Miles,50 Yards,VIRGINIA,38.9196,-77.2259,0
2,EF0,2022,0.05 Miles,30 Yards,VIRGINIA,38.8653,-77.4579,0
3,EF1,2022,9.14 Miles,500 Yards,PENNSYLVANIA,41.16,-76.73,1
4,EF1,2022,0.22 Miles,100 Yards,NORTH CAROLINA,35.8672,-78.8372,0
...,...,...,...,...,...,...,...,...
15844,EF1,2010,0.79 Miles,150 Yards,TEXAS,31.5095,-93.9232,0
15845,EF0,2010,0.16 Miles,25 Yards,LOUISIANA,30.815,-92.0455,0
15846,EF1,2010,4.62 Miles,25 Yards,CALIFORNIA,33.716,-118.125,0
15847,EF0,2010,0.14 Miles,10 Yards,CALIFORNIA,34.4248,-119.877,0


In [19]:
df = convertYardsToMeters(df)
df = convertMilesToKm(df)
df = convertScaleToNumbers(df)
df.rename(columns={'Scale': 'Scale (EF)', 'Length': 'Length (KM)', 'Width': 'Width (KM)'}, inplace=True)
df.to_csv('TornadoTableAfterDataCleaning.csv', index=False, header=True)

**The df after data cleaning:**

In [20]:
df = pd.read_csv("TornadoTableAfterDataCleaning.csv")
df

,Scale (EF),Years,Length (KM),Width (KM),Country,Latitude,Longtitude,Deaths
0,1,2022,3.235,0.091,PENNSYLVANIA,40.3790,-75.2266,0
1,0,2022,0.209,0.046,VIRGINIA,38.9196,-77.2259,0
2,0,2022,0.080,0.027,VIRGINIA,38.8653,-77.4579,0
3,1,2022,14.709,0.457,PENNSYLVANIA,41.1600,-76.7300,1
4,1,2022,0.354,0.091,NORTH CAROLINA,35.8672,-78.8372,0
...,...,...,...,...,...,...,...,...
14785,1,2010,1.271,0.137,TEXAS,31.5095,-93.9232,0
14786,0,2010,0.257,0.023,LOUISIANA,30.8150,-92.0455,0
14787,1,2010,7.435,0.023,CALIFORNIA,33.7160,-118.1250,0
14788,0,2010,0.225,0.009,CALIFORNIA,34.4248,-119.8770,0
